In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from requests_html import HTMLSession, AsyncHTMLSession

In [3]:
rejected = [
    'wa.link',
    'mailto:',
    't.me',
    'tally.so',
    'discord.gg',
    'arxiv.org',
    'javascript:'
]

In [4]:
async def get_html(url):
    session = AsyncHTMLSession(verify = False)
    result = None
    try:
        
        r = await session.get(url, verify = False, timeout = 10)
        await r.html.arender(timeout = 10)
        
        result = r.html.html
    except Exception as e:
        pass
    
    await session.close()
    return result


def get_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    return links

In [5]:
import asyncio

visited_pages = set(['https://www.mohr.gov.my/index.php/ms/'])

async def crawl(base_url):
    if base_url.endswith('.pdf'):
        return
    
    html = await get_html(base_url)
    
    if html is None:
        return

    visited_pages.add(base_url)
    links = get_links(html)
    for link in links:
        try:
            if any([r in link for r in rejected]):
                continue

            if not link.startswith('http'):
                link = urljoin(base_url, link)

            link = link.split('#')[0]
            link = link.split('?')[0]

            if link not in visited_pages:
                visited_pages.add(link)
        except Exception as e:
            pass

In [6]:
from tqdm import tqdm

batch_size = 10

for no in range(3):
    l = list(visited_pages)
    print(no, len(l))
    for i in tqdm(range(0, len(l), batch_size)):
        b = l[i: i + batch_size]
        await asyncio.gather(*[crawl(url) for url in b])

0 1


100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.70s/it]


1 97


100%|███████████████████████████████████████████| 10/10 [00:59<00:00,  5.93s/it]


2 2705


 42%|█████████████████▍                       | 115/271 [13:55<18:27,  7.10s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.sendMessageToTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.sendMessageToTarget: Target closed.
 66%|███████████████████████████▏             | 180/271 [22:11<08:03,  5.32s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
 90%|████████████████████████████████████▉    | 244/271 [32:16<04:09,  9.24s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retriev

In [8]:
import json

with open('url-mohr.gov.my.json', 'w') as fopen:
    json.dump(list(visited_pages), fopen)